# Quicksort

In [1]:
:set -XNoMonadFailDesugaring
:l SortersM
:l Predicates

What is unique about our method applied to quicksort is that by default we get not only multiple occurences of the same permutation, but also much more sequences:

In [2]:
quickSortM coinCmp [3,2,1] :: [[Int]]

[[1,2,3],[1,2,3,1],[1,2,3,2],[1,2,3,1,2],[1,2,3,1,2,1],[1,2,3,2],[1,2,3,2,1],[1,2,1,3],[1,2,1,3,1],[1,2,1,3,2],[1,2,1,3,1,2],[1,2,1,3,1,2,1],[1,2,1,3,2],[1,2,1,3,2,1],[2,3],[2,3,1],[2,3,2],[2,3,1,2],[2,3,1,2,1],[2,3,2],[2,3,2,1],[2,1,3],[2,1,3,1],[2,1,3,2],[2,1,3,1,2],[2,1,3,1,2,1],[2,1,3,2],[2,1,3,2,1],[2,3],[2,3,1],[2,3,2],[2,3,1,2],[2,3,1,2,1],[2,3,2],[2,3,2,1],[1,3],[1,3,1],[1,3,2],[1,3,1,2],[1,3,1,2,1],[1,3,2],[1,3,2,1],[3],[3,1],[3,2],[3,1,2],[3,1,2,1],[3,2],[3,2,1]]

Here, even predicate consistency doesn't help, but at least it removes sequences that are not valid permutations:

In [3]:
import Control.Monad.State
evalStateT (bubbleSortM consistentCoinCmp [3,2,1]) noChoices :: [[Int]]

[[3,2,1],[2,3,1],[2,1,3],[3,1,2],[3,2,1],[1,3,2],[1,2,3]]

The thing we should concentrate on is the way we split elements in the list with respect to the pivot. By default we perform two independent filterings. This results in a wide range of different element placements:

In [4]:
type Partition a m = (a -> m Bool) -> [a] -> m ([a], [a])
filterTwiceNDM :: (Monad m) => Partition a m
filterTwiceNDM p xs = do
    xs1 <- filterM p xs
    xs2 <- filterM (fmap not . p) xs
    return (xs1, xs2)

filterTwiceNDM (coinCmp 0) [1,2] :: [([Int], [Int])]

[([1,2],[]),([1,2],[2]),([1,2],[1]),([1,2],[1,2]),([1],[]),([1],[2]),([1],[1]),([1],[1,2]),([2],[]),([2],[2]),([2],[1]),([2],[1,2]),([],[]),([],[2]),([],[1]),([],[1,2])]

We might face it with either predicate consistency:

In [5]:
evalStateT (filterTwiceNDM (consistentCoinCmp 0) [1,2]) noChoices :: [([Int], [Int])]

[([1,2],[]),([1],[2]),([2],[1]),([],[1,2])]

or with a single-scan partitioning:

In [6]:
partitionNDM :: (Monad m) => Partition a m
partitionNDM _ [] = return ([], [])
partitionNDM p (x:xs) = do
    b <- p x
    (xs1, xs2) <- partitionNDM p xs
    return $ if b then (x:xs1, xs2) else (xs1, x:xs2)

partitionNDM (coinCmp 0) [1,2] :: [([Int], [Int])]

[([1,2],[]),([1],[2]),([2],[1]),([],[1,2])]

This is sufficient to obtain required result:

In [7]:
quickSortM :: (Monad m) => SorterM a m
quickSortM _ [] = return []
quickSortM p (x:xs) = do
    let fp = flip p
    (less, greater) <- partitionNDM (fp x) xs
    less' <- quickSortM p less
    greater' <- quickSortM p greater
    return (less' ++ [x] ++ greater')
    
quickSortM coinCmp [3,2,1] :: [[Int]]

[[1,2,3],[2,1,3],[2,3,1],[1,3,2],[3,1,2],[3,2,1]]